In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
def read_out_tr(dir, file):
    col = ["events", "time", "from_node", "to_node", "pkt_type", "pkt_size", "flags", "fid", "src_addr", "dst_addr", "seq_num", "pkt_id"]
    df = pd.read_csv(dir+file, sep = ' ', header = None)
    df.columns = col
    return df

# Monitoring queue length by Mengqi
def get_src_packets(df, switch):
    # get all the tcp pkts (Modified by Peixuan)
    df = df.loc[df["events"].isin(["+", "-"])]
    df = df.loc[df["pkt_type"].isin(["tcp"])]
    df = df.loc[df["from_node"].isin([switch])]
    return df

def sort_by_time(df):    
    df = df.sort_values(by=["time", "events"])
    return df

def plot(df):
    plt.figure(figsize=(18,5))
    #plt.plot(x,y,"b--",linewidth=1)   #在当前绘图对象绘图（X轴，Y轴，蓝色虚线，线宽度）
    plt.xlabel("Time")
    plt.ylabel("Queue Length")
    #plt.xscale('log')
    plt.title("Real Time Queue Length")    
    #line0 = plt.plot(df["time"][200:500], df["queue_len"][200:500], color='red', markersize = 12)
    line0 = plt.plot(df["time"], df["queue_len"], color='red')

    plt.legend(loc='best')
    plt.show()

def cal_queue_len(df_out_tr, switch):
    df_queue = get_src_packets(df_out_tr, switch)
    df_queue = sort_by_time(df_queue)
    pktCount = 0
    queue_len = []
    for index, row in df_queue.iterrows():
        if row["events"] == '-': 
            pktCount -= 1
            queue_len.append(pktCount)
        if row["events"] == '+':
            pktCount += 1
            queue_len.append(pktCount)
    df_queue["queue_len"] = queue_len
    plot(df_queue)
    
    #return df_sort
    